In [8]:
import pandas as pd
import numpy as np

In [45]:
PATH="../dat/textbooks/"
THRESHOLD = 2

In [18]:
df_de3 = pd.read_json(PATH+"de3.json")
adj_concepts = np.load(PATH+"concepts.npy")
adj_matrix = np.load(PATH+"adj.npy")

In [47]:
pruned = 0
total_deps = 0
for i, concept in enumerate(adj_concepts):
    delete_deps = set(adj_concepts[adj_matrix[i,:]<=10-THRESHOLD])
    proposed_deps = [d[3] for d in df_de3.loc[df_de3.concept==concept].iloc[0].dep_articles]
    pruned_deps = set(proposed_deps).intersection(delete_deps)
    total_deps += len(proposed_deps)
    if len(pruned_deps):
        pruned += len(pruned_deps)
        #print(f"on {concept} pruning " + str(pruned_deps)
        
print(f"{pruned}/{total_deps} are to be pruned")

450/6369 pruned


In [63]:
def prune_deps(concept, deps):
    i = np.where(adj_concepts==concept)[0][0]
    delete_deps = set(adj_concepts[adj_matrix[i,:]<=10-THRESHOLD])
    return list(set(deps).difference(delete_deps))
    

In [64]:
df_concepts = df_de3.copy()
df_concepts.dep_articles = df_concepts.apply(lambda r: prune_deps(r['concept'], [d[3] for d in r['dep_articles']]), axis=1)

In [66]:
df_concepts.to_json(PATH+"graph.json")
df_concepts.to_csv(PATH+"df_concepts.csv",index=False)